In [25]:
# global constants
g_maxAngVel = pi / 2 # move in azimuth or elevation at max rate of 90 deg / sec
g_maxScanRate = 1000 # scans points at the rate of 1000 pts/sec

# function to convert points from cartesian Part Coordiante System to cartesian Machine Coordinate System 
# input:  mcs: 4x4 transformation matrix
#         pts: nx3 list of points in PCS
# output: nx3 list of points in MCS
function pcs_to_mcs_xyz(mcs, pts)
    # transform all points into LR Cartesian coordinates
    # homogenize the pts with a 1
    qty = size(pts,1)
    tmp = [pts ones(qty,1)]
    # multiply by inverse of the LR's location/orientation
    ret = inv(mcs) * tmp'
    return ret[1:3,:]'
end;

# function to convert points from cartesian Part Coordiante System to cartesian Machine Coordinate System 
# input:  mcs: 4x4 transformation matrix
#         pts: nx3 list of points in MCS
# output: nx3 list of points in PCS
function mcs_to_pcs_xyz(mcs, pts)
    # transform all points into LR Cartesian coordinates
    # homogenize the pts with a 1
    qty = size(pts,1)
    tmp = [pts ones(qty,1)]
    # multiply by inverse of the LR's location/orientation
    ret = mcs * tmp'
    return ret[1:3,:]'
end;

# function to convert points from cartesian to spherical
# input:  pts: nx3 list of xyz (cartesian) points
# output: nx3 list of spherical points (range, theta, phi)
function xyz_to_sph(pts)
    r = [vecnorm(pts[i,:]) for i=1:size(pts,1)]
    r2= [vecnorm(pts[i,1:2]) for i=1:size(pts,1)]
    t = atan2.(r2, pts[:,3])
    p = atan2.(pts[:,2], pts[:,1])
    return [r t p]
end;

# function to convert points from spherical to cartesian
# input:  pts: nx3 list of (r,t,p) (spherical) points
# output: nx3 list of cartesian points (xyz)
function sph_to_xyz(pts)
    # multiply by inverse of the LR's location/orientation
    x = pts[:,1] .* sin.(pts[:,2]) .* cos.(pts[:,3]) # rsinθcosϕ
    y = pts[:,1] .* sin.(pts[:,2]) .* sin.(pts[:,3]) # rsinθsinϕ
    z = pts[:,1] .* cos.(pts[:,2])                   # rcosθ
    return [x y z]
end;


# function to simulate a scan from one spherical coord to another
# input: from: spherical coordinate triple (radius, theta, phi) to start from
#        to:   spherical coordinate triple (radius, theta, phi) to end at
# output: n x 3 list of spherical points
function scan_from_to(from, to)
    delta = to .- from
    time = maximum(abs.(delta[2:3])) / g_maxAngVel
    qty = floor(time * g_maxScanRate)
    rgs = linspace.(from, to, qty)
    return [collect(rgs[1]) collect(rgs[2]) collect(rgs[3])]
end

# function to simulate a scan through a list of (spherical) points
# input:  list of points (spherical) in the order to be visited
# output: list of points (spherical) of the simulated scan points
function scan_tour(pts)
    ret = []
    for i = 1:size(pts,1)-1
        line = scan_from_to(pts[i,:], pts[i+1,:])
        ret = [ret; line]
    end
    return ret
end

function test_transforms()
    # load the feature file
    data = readcsv("50_Holes_FrontLeft.csv");
    pts_pcs = convert(Array{Float64, 2}, data[:,2:4]) # feature locations in PCS
    lr_tfm = readcsv("Location1.csv") # transformation matrix for LR's position

    # convert to spherical
    pts_mcs = pcs_to_mcs_xyz(lr_tfm, pts_pcs) # convert feature locations to MCS
    pts_lr = xyz_to_sph(pts_mcs) # convert feature locations to spherical

    # round trip as a test
    pts_mcs_back = sph_to_xyz(pts_lr)
    pts_pcs_back = mcs_to_pcs_xyz(lr_tfm, pts_mcs_back)

    # points should be the same
    diff = pts_pcs - pts_pcs_back
    display(maximum(diff))
end

function test_scan()
    data = readcsv("50_Holes_FrontLeft.csv");
    pts_pcs = convert(Array{Float64, 2}, data[:,2:4]) # feature locations in PCS
    lr_tfm = readcsv("Location1.csv") # transformation matrix for LR's position
    sph = xyz_to_sph(pcs_to_mcs_xyz(lr_tfm, pts_pcs))
    scan = scan_tour(sph)
    scan = mcs_to_pcs_xyz(lr_tfm, sph_to_xyz(scan))
    println(size(scan))
end

test_transforms()
test_scan()

8.881784197001252e-16

(4498, 3)
